# **TUGAS 1 SELEKSI ASISTEN LAB BASIS DATA 2023**
#Scraping IMDb Website to Get Top 250 Movies of All Time
#Josua Adriel Sinabutar     
#18221065

#1. Import All Libraries that we need

In [5]:
import time
import requests
from bs4 import BeautifulSoup #here i used beautiful soup because its the one that is very familiar for me
import json # we want to dump the data in json file type


#2. Create the Class of Film Data

In [12]:
class FilmData:
    def __init__(self, rank, title, year_created, duration, classification, rating):
        self.rank = rank
        self.title = title
        self.year_created = year_created
        self.duration = duration
        self.classification = classification
        self.rating = rating

    def cetakKeLayar(self):
        print(f'''
        Rank : {self.rank}
        Film Title : {self.title}
        Year Created: {self.year_created}
        Film Duration: {self.duration}
        Film Classification: {self.classification}
        Film IMDb Rating: {self.rating}
        ''') #multiline string literal in Python
    def getYear(self):
        return self.year_created
    def getDuration(self):
        return self.duration
    def getClassification(self):
        return self.classification
    def setYear(self, year):
        self.year_created = year
    def setDuration(self, duration):
        self.duration = duration
    def setClassification(self, classification):
        self.classification = classification

#3. Write the Function that Actually Take the Data from IMDb Web

In [13]:
def scrapeTheWeb(filmList):
        url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250" #this is our url source
        headers = {
            'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        }#I introduce myself so that i am identified as browser not bot. I tried to state my email, but it didn't work
        req = requests.get(url, headers=headers)
        print(req) #printing the response
        soup = BeautifulSoup(req.text, 'html.parser') #using html parser just because my reference use it :)
        films = soup.findAll('li', 'ipc-metadata-list-summary-item sc-bca49391-0 eypSaE cli-parent')
        error = False #error identifier
        print('Scraping...')
        for film in films:
            try:
                title_with_rank = film.find('h3', 'ipc-title__text').text
                rank = title_with_rank.split(".")[0].strip()#getting the rank from the title numbering
                title = title_with_rank.split(".")[1].strip()# getting only the film title
                descriptions = film.find('div', class_='sc-14dd939d-5 cPiUKY cli-title-metadata') #the year, duration, and classification are united in one div so i had to take them all together, put it on list, then store it to different variables
                if descriptions:
                    each = descriptions.find_all('span', class_='sc-14dd939d-6 kHVqMR cli-title-metadata-item')
                    each_values = [span.text for span in each]
                else:
                    each_values = []
                year_created = each_values[0] if len(each_values) >= 1 else '9999' #get the year data
                duration = each_values[1] if len(each_values) >= 2 else '9999' #get the duration data
                classification = each_values[2] if len(each_values) == 3 else 'N/A' #get the classification data
                rating = film.find('span', 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').text #get IBDb users rating regarding each film
                completeFilm = FilmData(rank, title, year_created, duration, classification, rating) #append all data to a complete data for each film
                filmList.append(completeFilm) #append it to the film list
                time.sleep(2)#waiting time to keep the "ethics"
            except:
                print("Scraping Error!") #incase the scraping error
                error = True
        if not error:
            print("Scraping is finished smoothly! Congrats.")

# **#4. Scraping imdb.com**

In [15]:
result = []
scrapeTheWeb(result)
print(len(result))#check whether it contains 250 films

<Response [200]>
Scraping...
Scraping is finished smoothly! Congrats.
250


#5. Data Preprocessing

We try to find if there are some invalid data in the result list. I have set the 'N/A' value if there is nothing to take from the web. Let's check it out.

In [16]:
durationError = 0
yearError = 0
classificationError = 0
for t in result:
    if (FilmData.getYear(t) == 'N/A'):
        yearError += 1
    elif (FilmData.getDuration(t) == 'N/A'):
        durationError += 1
    elif (FilmData.getClassification(t) == 'N/A'):
        classificationError += 1
print(durationError)
print(yearError)
print(classificationError)

0
0
1


We found ***one*** error in classification data. Since it is very minor error, I decided to classify that film into 'R' category. This is how i did it.

In [17]:
for t in result:
    if (FilmData.getClassification(t) == 'N/A'):
        FilmData.setClassification(t, '18+')
        FilmData.cetakKeLayar(t) #to proof that it has been changed

 
        Rank : 77
        Film Title : Das Boot
        Year Created: 1981
        Film Duration: 2h 29m
        Film Classification: 18+
        Film IMDb Rating: 8.4
        


**Voila!** The error data has been fixed. Now all of the data is valid and we are ready to dump it.

#5. Dump the Data Into JSON Format

In [18]:
with open('IMDb 250 Top Movies.json', 'w') as f:
        json.dump([obj.__dict__ for obj in result], f, indent=4)

# **FIN**